In [3]:
import sys
import pyspark.sql
from pyspark.sql import functions as F


In [6]:
spark = SparkSession.builder.appName("Modulo 3 Trabalho Pratico").getOrCreate()

24/05/25 11:47:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
basics = '/home/telma/xpe/engenharia_dados/title_basics.tsv'
ratings = '/home/telma/xpe/engenharia_dados/title_ratings.tsv'

In [8]:
df_titles = spark.read.csv(basics, header=True, inferSchema=True, sep='\t')

In [10]:
df_ratings = spark.read.csv(ratings, header=True, inferSchema=True, sep='\t')

In [13]:
x = df_titles.count()
print(f"Arquivo basics possui {x} registros.")

Arquivo basics possui 8203690 registros.


In [14]:
y = df_ratings.count()
print(f"Arquivo ratings possui {y} registros.")

Arquivo ratings possui 1182639 registros.


In [18]:
# Dataframe que relaciona basics e ratings
df_join = df_titles.join(df_ratings,"tconst", "left_outer")

In [19]:
df_join.show(6, False)

+----------+---------+-------------------------------------+-------------------------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|tconst    |titleType|primaryTitle                         |originalTitle                        |isAdult|startYear|endYear|runtimeMinutes|genres         |averageRating|numVotes|
+----------+---------+-------------------------------------+-------------------------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|tt0000002 |short    |Le clown et ses chiens               |Le clown et ses chiens               |0      |1892     |\N     |5             |Animation,Short|6.0          |233     |
|tt0000004 |short    |Un bon bock                          |Un bon bock                          |0      |1892     |\N     |12            |Animation,Short|6.1          |152     |
|tt11106020|tvEpisode|Mästarna är tillbaka!                |Mästarna är tillbaka!                |0      

In [25]:
df_titles.select("titleType").distinct().sort("titleType").show(15)

+------------+
|   titleType|
+------------+
|       movie|
|radioEpisode|
| radioSeries|
|       short|
|   tvEpisode|
|tvMiniSeries|
|     tvMovie|
|     tvPilot|
|    tvSeries|
|     tvShort|
|   tvSpecial|
|       video|
|   videoGame|
+------------+



In [31]:
df_titles.filter("lower(titleType) like ('%movie') and startYear = 2015").count()

19987

In [42]:
df_split = df_join.withColumn("genre_exp", F.split("genres", ",", 3))

In [43]:
df_split.show(6, False)

+----------+---------+-------------------------------------+-------------------------------------+-------+---------+-------+--------------+---------------+-------------+--------+------------------+
|tconst    |titleType|primaryTitle                         |originalTitle                        |isAdult|startYear|endYear|runtimeMinutes|genres         |averageRating|numVotes|genre_exp         |
+----------+---------+-------------------------------------+-------------------------------------+-------+---------+-------+--------------+---------------+-------------+--------+------------------+
|tt0000002 |short    |Le clown et ses chiens               |Le clown et ses chiens               |0      |1892     |\N     |5             |Animation,Short|6.0          |233     |[Animation, Short]|
|tt0000004 |short    |Un bon bock                          |Un bon bock                          |0      |1892     |\N     |12            |Animation,Short|6.1          |152     |[Animation, Short]|
|tt1110602

In [59]:
df_genero = df_split.select("*").withColumn("genre", F.explode(df_split.genre_exp))

In [60]:
df_genero.show(5, False)

+---------+---------+--------------------------------------+--------------------------------------+-------+---------+-------+--------------+-----------------+-------------+--------+--------------------+-----------+
|tconst   |titleType|primaryTitle                          |originalTitle                         |isAdult|startYear|endYear|runtimeMinutes|genres           |averageRating|numVotes|genre_exp           |genre      |
+---------+---------+--------------------------------------+--------------------------------------+-------+---------+-------+--------------+-----------------+-------------+--------+--------------------+-----------+
|tt0000008|short    |Edison Kinetoscopic Record of a Sneeze|Edison Kinetoscopic Record of a Sneeze|0      |1894     |\N     |1             |Documentary,Short|5.5          |1965    |[Documentary, Short]|Documentary|
|tt0000008|short    |Edison Kinetoscopic Record of a Sneeze|Edison Kinetoscopic Record of a Sneeze|0      |1894     |\N     |1             |

In [62]:
# Gẽnero de títulos mais frequente
df_genero.groupBy("genre").agg(F.count("genre").alias("QTD")).orderBy("QTD", ascending = 0).show(5)

24/05/25 12:58:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/25 12:58:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+-------+
|      genre|    QTD|
+-----------+-------+
|      Drama|2247995|
|     Comedy|1653725|
|      Short|1021850|
|  Talk-Show| 900198|
|Documentary| 764885|
+-----------+-------+
only showing top 5 rows



In [63]:
# Gênero com a melhor nota média
df_genero.groupBy("genre").agg(F.mean("averageRating").alias("Rating")).orderBy("Rating", ascending = False).show(5, False)

24/05/25 13:00:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/25 13:00:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+-----------------+
|genre      |Rating           |
+-----------+-----------------+
|History    |7.353780102645096|
|Documentary|7.240198535554567|
|Biography  |7.175531914893609|
|Mystery    |7.170086406897949|
|Crime      |7.159842868485928|
+-----------+-----------------+
only showing top 5 rows



In [65]:
df_genero.filter("genre='Adventure' and titleType = 'videoGame' and startYear = 2020")\
        .orderBy("averageRating", ascending = False)\
        .show(5, False)

+----------+---------+-----------------------------+-----------------------------+-------+---------+-------+--------------+------------------------+-------------+--------+----------------------------+---------+
|tconst    |titleType|primaryTitle                 |originalTitle                |isAdult|startYear|endYear|runtimeMinutes|genres                  |averageRating|numVotes|genre_exp                   |genre    |
+----------+---------+-----------------------------+-----------------------------+-------+---------+-------+--------------+------------------------+-------------+--------+----------------------------+---------+
|tt11321196|videoGame|Half-Life: Alyx              |Half-Life: Alyx              |0      |2020     |\N     |\N            |Action,Adventure,Horror |9.5          |506     |[Action, Adventure, Horror] |Adventure|
|tt7651352 |videoGame|Ghost of Tsushima            |Ghost of Tsushima            |0      |2020     |\N     |\N            |Action,Adventure,Drama  |9.3     

In [66]:
df_titles.select('primaryTitle').distinct().count()

3931670

In [68]:
df_titles.filter("isAdult = 1").describe("runtimeMinutes").show()

+-------+-----------------+
|summary|   runtimeMinutes|
+-------+-----------------+
|  count|           250127|
|   mean|92.79938555059914|
| stddev|57.18982244754778|
|    min|                1|
|    max|               \N|
+-------+-----------------+



In [69]:
df_titles.filter("primaryTitle <> originalTitle").count()

125056

In [71]:
df_titles.orderBy(F.length("primaryTitle").desc()).show(5, False)

+----------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+---------+-------+--------------+-----------+
|tconst    |titleType|primaryTitle                                                  

In [72]:
df_join.select("tconst", "numVotes").orderBy("numVotes", ascending = False).show(5, False)

+---------+--------+
|tconst   |numVotes|
+---------+--------+
|tt0111161|2449517 |
|tt0468569|2405191 |
|tt1375666|2157649 |
|tt0137523|1930108 |
|tt0110912|1898801 |
+---------+--------+
only showing top 5 rows



In [76]:
df_join.describe("averageRating").show()

+-------+-----------------+
|summary|    averageRating|
+-------+-----------------+
|  count|          1182639|
|   mean|6.917028357766534|
| stddev|1.397496457577608|
|    min|              1.0|
|    max|             10.0|
+-------+-----------------+

